# Redes Neuronales

Una red neuronal es un clasificador. Su funcionamiento se basa en la combinación de hiperplanos que acotan regiones de activación del espacio de muestras.


### Mini-batch

En cada ciclo de actualización de los pesos de la red no es necesario calcular el gradiente de descenso para cada muestra. Basta con escoger un lote variado de muestras (mini-batch) y calcular el gradiente resultante de éste. 

### Loss

La pérdida o *loss* es el valor o error resultante de comparar las predicciones de la red con las etiquetas de las muestras.

### One-hot

Cuando se clasifica en más de dos categorías es necesario que la red tenga tantas neuronas en la capa de salida como clases haya. Por tanto, la activación de cada una de estas neuronas de salida corresponderá con una clase. Por ejemplo, si la red tiene cuatro salidas y se activa la tercera, el vector resultante podría ser algo como: [0.002,0.008,0.95,0.04]. Como normalmente las etiquetas de las muestras están en un formato numérico entero (0,1,2,3...), no podremos realizar la comparación de las salidas con las etiquetas. Así que transformamos la etiqueta en un vector de todo ceros excepto el lugar correspondiente a la etiqueta, el cual tendrá un valor de 1. Por ejemplo: $2\rightarrow [0,0,1,0]$. (Recordemos que las etiquetas comienzan en 0). Esta función de transformación se denomina **One-hot**.

### Época

Cada vez que entrenamos una red con todos los lotes o *mini-batches* de nuestro dataset de entrenamiento decimos que hemos completado una **época**. Durante el proceso de entrenamiento de una red se suelen completar múltiples épocas.

### Softmax

La función softmax es parecida a la sigmoide, ésta también convierte un valor entre menos infinito y más infinito en un valor entre cero y uno. La diferencia está en que la función softmax no trabaja sobre un valor sino sobre un vector. De esta forma, convierte todas las componentes de un vector en valores entre cero y uno, pero, además, garantiza que la suma de todos estos valores sea uno. La función softmax se emplea en la capa de salida de una red, y convierte de esta forma sus salidas en una distribución de probabilidades.

$$ Softmax(\textbf{v})_i = \frac{e^{v_i}}{\sum_{j=1}^{n} e^{v_j}} $$


## Dataset Iris

El dataset iris corresponde a un conjunto de 150 muestras pertenecientes a 3 categorías de flor del iris distintas (versicolor, virgínica y setosa). Estas 3 especies se pueden diferenciar por el ancho y alto de su pétalo y su sépalo. Vamos a construir una red que nos permita clasificar este dataset. Para ello utilizaremos 5 neuronas en una capa oculta y 3 neuronas en la capa de salida, ya que son 3 clases. 



https://keras.io/


Keras es una API de redes neuronales de alto nivel, escrita en Python y capaz de ejecutarse sobre TensorFlow, CNTK o Theano. Fue desarrollado con la idea de permitir la experimentación rápida, poder pasar de la idea al resultado en el menor tiempo posible.

Instala <a href="https://www.tensorflow.org/install">Tensorflow</a> y <a href="https://keras.io/#installation">Keras</a> y veamos cómo montar una red en Keras para clasificar el dataset Iris. En primer lugar importamos el dataset. La función `LabelBinarizer` nos convertirá las etiquetas numéricas enteras a codificación one-hot. La función *train_test_split()* nos divide el dataset con un 75% para el entrenamiento y un 25% para el test. 

In [0]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

#load the iris dataset
iris = load_iris()

X = iris.data
encoder = LabelBinarizer()
y = encoder.fit_transform(iris.target)  # We transform to one-hot
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Creamos la red con la capa de entrada correspondiente a los cuatro valores de cada muestra. Una capa oculta de cinco neuronas y una capa de salida de tres neuronas. Aplicamos la función de activación *sigmoide* para la capa oculta y, dado que tenemos más de dos clases, softmax para la salida. Utilizamos Stochastic Gradient Descent como método optimizador con un learning rate de 0.1.

In [0]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD #Stochastic Gradient Descent Optimizer

inputs = Input(shape=(4,))  # We have four inputs
x = Dense(5, activation='sigmoid')(inputs)  # five neurons and sigmoid as activation function
predictions = Dense(3, activation='softmax')(x)  # Output layer with three neurons, one for each class

# This creates a model
model = Model(inputs=inputs, outputs=predictions)
sgd = SGD(lr=0.1)
model.compile(optimizer=sgd,
              loss='mse',  # Mean Squared Error
              metrics=['accuracy'])

print(model.summary())

Ahora ya solo queda entrenar nuestra red y ver qué resultados nos devuelve.

In [0]:
model.fit(X_train, y_train, epochs=500)  # starts training with 500 epochs

Vamos a ver qué valores de salida tenemos para algunas muestras.

In [0]:
predictions = model.predict(X_test)
for p, l in zip(predictions, y_test):
    print(p,"->", l)

# Ejercicio práctico

Basándote en el ejercicio anterior, construye una red que clasifique el conjunto MNIST. En el siguiente código ya tenemos el dataset preparado para introducirlo en nuestra red. Recuerda que el conjunto de entrenamiento del MNIST está formado por 60000 imágenes de dígitos manuscritos de 28x28 pixels.

En este caso, se ha decidido no modificar el código que se tenía ya preparado ni ninguno de sus parámetros iniciales (batch_size, epochs...):

In [0]:
from keras.datasets import mnist
import keras

batch_size = 128
num_classes = 10
epochs = 5

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)  # 28x28=784
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255  # We normalize to have values between 0 and 1
X_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)  # onehot encoding
y_test = keras.utils.to_categorical(y_test, num_classes)

Creamos la red con la capa de entrada correspondiente a los 784 valores de cada muestra. Tres capas ocultas (De 980, 500 y 17 neuronas) y una capa de salida de 10 neuronas. Aplicamos la función de activación 'relu' (Parecida a una rampa si x > 0) para las capas ocultas y, dado que tenemos más de dos clases, softmax para la salida. Utilizamos RMSProp como método optimizador con un LR adaptativo y medimos la pérdida con Categorical Crossentropy.

In [0]:
# To do...
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD #Stochastic Gradient Descent Optimizer

inputs = Input(shape=(784,))  # We have 784 inputs
x = Dense(1000, activation='relu')(inputs)  # 1000 neurons and relu as activation function
y = Dense(500, activation='relu')(x) # 500 neurons and relu as activation function
z = Dense(25, activation='relu')(y)  # 25 neurons and relu as activation function
predictions = Dense(num_classes, activation='softmax')(z)  # Output layer with 10 neurons, one for each class

# This creates a model
model = Model(inputs=inputs, outputs=predictions)

# Instead of using MSE as Loss & SGD as Optimizer, I used Categorical Crossentropy & RMSProp to get adaptive LR and better results.
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',  # Categorical Crossentropy
              metrics=['accuracy'])

print(model.summary())

Ahora ya solo queda entrenar nuestra red y ver qué resultados nos devuelve.

In [0]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)  # starts training with 5 epochs

Vamos a ver qué valores de salida tenemos para algunas muestras. Además, se intenta sacar numéricamente cuántos resultados han sido positivos y cuántos han sido negativos. Ya que el set de 'Test' se encuentra separado al de 'Train', se puede visualizar si se ha realizado overfitting o no.

In [0]:
predictions = model.predict(X_test)
import numpy
correct = 0
incorrect = 0
for p, l in zip(predictions, y_test):
    if l[numpy.where(p == p.max())] == 1:
      correct += 1
    else:
      incorrect += 1
    print(p,"->", l)
print("Aciertos = ", correct)
print("Fallos = ", incorrect)